<a href="https://colab.research.google.com/github/MK316/workspace/blob/main/ASR01/ASR04_RecRate_SE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recognition Rate calculation using nltk

Need a recognition result file and rainbow sentence file

* **SE.zip** (gTTS generated audio files) - completed (asof 0106), [SETR_result.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SETR_result.csv)
* **SK.zip** (gTTS generated audio files) - completed (asof 0106), [SKTR_result.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SKTR_result.csv)
* **SK.zip 2** with medium.en model [SKTR_result_m.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SK_result_m.csv)
* HE.zip (Native production)
* HK.zip (Learners' production)

## Perception rate ~ Error rate

* Perception rate: Percentage of words that match the original text

### Demo

- Sample text: "**The rainbow** is a division of white light into many beautiful colors." (12 words)

- Perceived text: **The lane bowl** is a division of white light into many beautiful colors. (11 words)

Perception rate = (11/12)*100 = 91%  
Error rate = 9%

### Read the rainbow text (19 sentences)

[The rainbow passage sentence file list from github](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentences.csv)

SR result file from github: [SK](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SK_result_0106.csv), [SE](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SE_result_0106.csv)

🌱 Variable: sk = result file after (1) the sentence recognition process (2) Runtime

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentences.csv"
rb = pd.read_csv(url)
print(rb.head())

url2 = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SK_result_m_0109.csv"
sk = pd.read_csv(url2)
sk.head()



   SN                                           Sentence
0  S1  When the sunlight strikes raindrops in the air...
1  S2  The rainbow is a division of white light into ...
2  S3  These take the shape of a long round arch, wit...
3  S4  There is, according to legend, a boiling pot o...
4  S5             People look, but no one ever finds it.


,Filename,Runtime,Recognized
0,SK01.wav,5.360,When the sunlight strikes raindrops in the ai...
1,SK02.wav,0.953,Ta rainbow EJ division of white light into ma...
2,SK03.wav,4.879,"T-z take the shape away long round arch, with..."
3,SK04.wav,1.000,"Theor is, according to legend, a building put..."
4,SK05.wav,9.779,Peepoh roop!


🌱 Variable: **sk**

💾 SKR_result.csv (sk = [['SN', 'Filename', 'Runtime', 'Sentence', 'Recognized']])

In [ ]:
# Add original text to the result file
oritext = list(rb['Sentence'])
sn = list(rb['SN'])

sk['Sentence'] = oritext
sk['SN'] = sn
sk.head()

sk = sk[['SN', 'Filename', 'Runtime', 'Sentence', 'Recognized']]
cols = list(sk.columns.values)
sk.to_csv('SKR_result.csv', index=False)
sk.head()

,SN,Filename,Runtime,Sentence,Recognized
0,S1,SK01.wav,5.360,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...
1,S2,SK02.wav,0.953,The rainbow is a division of white light into ...,Ta rainbow EJ division of white light into ma...
2,S3,SK03.wav,4.879,"These take the shape of a long round arch, wit...","T-z take the shape away long round arch, with..."
3,S4,SK04.wav,1.000,"There is, according to legend, a boiling pot o...","Theor is, according to legend, a building put..."
4,S5,SK05.wav,9.779,"People look, but no one ever finds it.",Peepoh roop!


## Recognition rate and finding mismatching words using NLTK

In [ ]:
#@markdown Calculate recognition rate, record missing words => dataframe (SKTR_result.csv) 
sk1=sk
from nltk.tokenize import RegexpTokenizer

# number of words in the sentence
nws = []
# number of words in the recognized text
nwr = []
# number of missing words 
nmw = []
# number of words actually recognized
nwar = []

# Recgonition Rate
rr = []
#
nr = []
# Missing word list
mw = []
# Correctly recognized word list
recword=[]


for i in range(0,len(sk['SN'])):
  t1 = sk['Sentence'][i]

# text 1
  txt1 = t1.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist = tokenizer.tokenize(txt1)
  nt = len(wlist)
  nws.append(nt)

# text 2
  t2 = sk['Recognized'][i]
  txt2 = t2.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist1 = tokenizer.tokenize(txt2)
  nt1 = len(wlist1)
  nwr.append(nt1)

# Recognition rate

  # from tables.idxutils import calc_chunksize
  # from nltk.downloader import ErrorMessage
# txt1(original text), txt2 (recognized text)
 
  mword = []
  rword = []
  score = 0
  for i in range(0, len(wlist1)):
      w = wlist1[i]

      if w in wlist:
        sc = 1
        rword.append(w)
      else:
        sc = 0
        mword.append(w)

      score = score + sc
      mwords = ', '.join(mword)
      rwords = ', '.join(rword)
  mw.append(mwords)
  missingword = round((score/len(wlist))*100,2)
  nr.append(score)
  recword.append(rwords)

  # RecRate = float(format(missingword, '.2f'))
  # ErrRate = float(format((100.0 - RecRate), '.2f'))
  
  rr.append(missingword)
  # print('Matching words: %d'%score, 'out of %d words'%len(wlist))
  # print("="*50)
  # print('Recognition Rate: %f %%'%RecRate)
  # print('Error Rate: %f %%'%ErrRate)

sk1['LenS'] = nws
sk1['LenR'] = nwr
sk1['N_RecW'] = nr
sk1['RecRate'] = rr
sk1['Recognized_words'] = recword
sk1['MissRec_words'] = mw

sk1.head()


,SN,Filename,Runtime,Sentence,Recognized,LenS,LenR,N_RecW,RecRate,Recognized_words,MissRec_words
0,S1,SK01.wav,5.360,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...,17,16,10,58.82,"when, the, sunlight, strikes, raindrops, in, t...","rays, of, sun, start, to, vanish"
1,S2,SK02.wav,0.953,The rainbow is a division of white light into ...,Ta rainbow EJ division of white light into ma...,12,11,9,75.00,"rainbow, division, of, white, light, into, man...","ta, ej"
2,S3,SK03.wav,4.879,"These take the shape of a long round arch, wit...","T-z take the shape away long round arch, with...",22,23,14,63.64,"take, the, shape, long, round, arch, with, its...","t, z, away, twinge, apparent, wouldn, t, it, p..."
3,S4,SK04.wav,1.000,"There is, according to legend, a boiling pot o...","Theor is, according to legend, a building put...",13,12,8,61.54,"is, according, to, legend, a, at, one, end","theor, building, putover, could"
4,S5,SK05.wav,9.779,"People look, but no one ever finds it.",Peepoh roop!,8,2,0,0.00,,"peepoh, roop"


💾 Column ordering & save it as csv (e.g., 'SKTR_result_m_0110.csv')

In [ ]:
sk2 = sk1[['SN', 'Filename', 'Runtime', 'Recognized','LenR','Sentence','LenS','N_RecW','RecRate','Recognized_words','MissRec_words']]
cols = list(sk2.columns.values)
sk2.to_csv('SKTR_result_m_0110.csv', index=False)
sk2.head()

,SN,Filename,Runtime,Recognized,LenR,Sentence,LenS,N_RecW,RecRate,Recognized_words,MissRec_words
0,S1,SK01.wav,5.360,When the sunlight strikes raindrops in the ai...,16,When the sunlight strikes raindrops in the air...,17,10,58.82,"when, the, sunlight, strikes, raindrops, in, t...","rays, of, sun, start, to, vanish"
1,S2,SK02.wav,0.953,Ta rainbow EJ division of white light into ma...,11,The rainbow is a division of white light into ...,12,9,75.00,"rainbow, division, of, white, light, into, man...","ta, ej"
2,S3,SK03.wav,4.879,"T-z take the shape away long round arch, with...",23,"These take the shape of a long round arch, wit...",22,14,63.64,"take, the, shape, long, round, arch, with, its...","t, z, away, twinge, apparent, wouldn, t, it, p..."
3,S4,SK04.wav,1.000,"Theor is, according to legend, a building put...",12,"There is, according to legend, a boiling pot o...",13,8,61.54,"is, according, to, legend, a, at, one, end","theor, building, putover, could"
4,S5,SK05.wav,9.779,Peepoh roop!,2,"People look, but no one ever finds it.",8,0,0.00,,"peepoh, roop"


The end

# 🌀Recognition Rate without stopwords (23.01.10~)

🌱 Variables: 

* sk2 = combined results
* sk (reduced for nltk process)= sentence, recognized texts

In [ ]:
sk = sk2[['SN','Sentence','Recognized']]
sk.head()

,SN,Sentence,Recognized
0,S1,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...
1,S2,The rainbow is a division of white light into ...,Ta rainbow EJ division of white light into ma...
2,S3,"These take the shape of a long round arch, wit...","T-z take the shape away long round arch, with..."
3,S4,"There is, according to legend, a boiling pot o...","Theor is, according to legend, a building put..."
4,S5,"People look, but no one ever finds it.",Peepoh roop!


**Variable: sw = nltk stopword list** (179 words)

💾 stopword list as csv

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

sw = stopwords.words('english')
print(stopwords.words('english'))
len(sw)

dfs = pd.DataFrame()
dfs['ID'] = range(1,180)
dfs['Stopwords'] = sw
dfs.to_csv('nltkstopwords.csv', index=False)
dfs.head()

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,ID,Stopwords
0,1,i
1,2,me
2,3,my
3,4,myself
4,5,we


Variable: sw (stopword list)

In [ ]:
# Checking
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

example_sent = sk2['Sentence'][0]
word_tokens = tokenizer.tokenize(example_sent)
stop_words = set(stopwords.words('english'))

filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#with no lower case conversion
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
 
print('Number of word tokens: %d'%len(word_tokens))
print(word_tokens)
print('='*50)
print('Number of words after stopwords: %d'%len(filtered_sentence))
print(filtered_sentence)

Number of word tokens: 17
['When', 'the', 'sunlight', 'strikes', 'raindrops', 'in', 'the', 'air', 'they', 'act', 'as', 'a', 'prism', 'and', 'form', 'a', 'rainbow']
Number of words after stopwords: 9
['When', 'sunlight', 'strikes', 'raindrops', 'air', 'act', 'prism', 'form', 'rainbow']


Variable: sk3 = sk2 for further process

In [ ]:
sk3 = sk2
sk3.head()

,SN,Filename,Runtime,Recognized,LenR,Sentence,LenS,N_RecW,RecRate,Recognized_words,MissRec_words,Sent_ws,Rec_ws
0,S1,SK01.wav,5.360,When the sunlight strikes raindrops in the ai...,16,When the sunlight strikes raindrops in the air...,17,10,58.82,"when, the, sunlight, strikes, raindrops, in, t...","rays, of, sun, start, to, vanish",When sunlight strikes raindrops air act prism ...,When sunlight strikes raindrops air rays sun s...
1,S2,SK02.wav,0.953,Ta rainbow EJ division of white light into ma...,11,The rainbow is a division of white light into ...,12,9,75.00,"rainbow, division, of, white, light, into, man...","ta, ej",The rainbow division white light many beautifu...,Ta rainbow EJ division white light many beauti...
2,S3,SK03.wav,4.879,"T-z take the shape away long round arch, with...",23,"These take the shape of a long round arch, wit...",22,14,63.64,"take, the, shape, long, round, arch, with, its...","t, z, away, twinge, apparent, wouldn, t, it, p...",These take shape long round arch path high two...,T z take shape away long round arch path high ...
3,S4,SK04.wav,1.000,"Theor is, according to legend, a building put...",12,"There is, according to legend, a boiling pot o...",13,8,61.54,"is, according, to, legend, a, at, one, end","theor, building, putover, could",There according legend boiling pot gold one end,Theor according legend building putover could ...
4,S5,SK05.wav,9.779,Peepoh roop!,2,"People look, but no one ever finds it.",8,0,0.00,,"peepoh, roop",People look one ever finds,Peepoh roop


In [ ]:
from pandas.io.formats.style_render import DataFrame
# Bulk process

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

# list variables to collect each result

sent_wsls = []
rec_wsls = []
rr_ws = []
corecwords = []
newrate = []
# Length of sentence without stopwords, recognized text without stopwords
lenS_sw = []
lenR_sw = []
sn = len(sk3['Sentence'])

# 1. removing stopwords in the original & recognized texts
for i in range(0,sn):
  ori_sent = sk3['Sentence'][i]
  rec_sent = sk3['Recognized'][i]
  word_tokens = tokenizer.tokenize(ori_sent)
  rword_tokens = tokenizer.tokenize(rec_sent)
  stop_words = set(stopwords.words('english'))

  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  rfiltered_sentence = [w for w in rword_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  rfiltered_sentence = []
 
  for w in word_tokens:
    if w not in stop_words:
       filtered_sentence.append(w)
  fs = ' '.join(filtered_sentence)
  sent_wsls.append(fs)

  for w in rword_tokens:
    if w not in stop_words:
       rfiltered_sentence.append(w)
  rfs = ' '.join(rfiltered_sentence)
  rec_wsls.append(rfs)

  correctwords = []
  score=0
  n1 = len(filtered_sentence)
  n2 = len(rfiltered_sentence)
  for k in range(0,n1):
      if filtered_sentence[k] in rfiltered_sentence:
        sc = 1
        correctwords.append(filtered_sentence[k])
      else:
        sc = 0 
      score = score + sc
  nwrt = round((score/n1)*100,2)
  newrate.append(nwrt)
  rr_ws.append(score)
  cwl = ', '.join(correctwords)
  corecwords.append(cwl)
  lenS_sw.append(n1)
  lenR_sw.append(n2)

#2 removing stopwords in the recognized words


# Fixed due to Copy warning
sk3['Sent_ws'] = sent_wsls
sk3['LenS_ws'] = lenS_sw
sk3['Rec_ws'] = rec_wsls
sk3['LenR_ws'] = lenR_sw
sk3['Correctwords_ws'] = corecwords
sk3['Recwords_ws'] = rr_ws
sk3['RecRate_ws'] = newrate
sk3.head()

# 3. writing the result in the DataFrame
sk3.to_csv('SKTR_SW_result.csv')